In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Usable Code

In [2]:
!pwd

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [3]:
files = sorted([f for f in os.listdir('csv') if '.csv' in f])

In [4]:
files

['07-08.csv',
 '08-09.csv',
 '09-10.csv',
 '10-11.csv',
 '11-12.csv',
 '12-13.csv',
 '13-14.csv',
 '14-15.csv',
 '15-16.csv',
 '16-17.csv',
 '17-18.csv',
 '18-19.csv']

In [5]:
def win_loss(score):
    if score > 0:
        return 'V'
    else:
        return 'H'

In [6]:
def clean_odds(file):
    '''
    cleans csv file and returns dataframe object that contains the over/under lines (target)
    '''
    df = pd.read_csv(file)
    df = df.iloc[:,:13] #some files contained unnamed extra rows
    df.dropna(axis=0,inplace=True) #and others columns
    
    #makes a list of game_id's equivalent to how many games were played that season
    mylist = []
    for i in range(1, int(df.shape[0]/2 + 1)):
        mylist.append(i)
        mylist.append(i)
    df['game_id'] = mylist #maps game_id's to games (spread across 2 rows)
    
    df = df.merge(df, on='game_id', suffixes=(None,'_2')) #gets games to be in one row (creates 4/game)
    df = df[1::4] #gets correct mapping
    df.reset_index(drop=True, inplace=True)
    
    #make dates include year so we can join with another dataset later
    df['Date'] = df['Date'].astype('int')
    df['Date'] = df['Date'].astype('str')
    #df['Date'] = df['Date'].str.strip('.0')
    dates = []
    for date in df['Date']:
        if int(date) > 1000:
            date = '20'+file[:2]+date #GET FILENAMES AND USE HERE
            dates.append(date)
        else:
            date = '20'+file[3:5]+'0'+date #GET FILENAMES AND USE HERE
            dates.append(date)
    df['Date'] = dates
    
    #take out pick-em's and replace with zero for later transformation
    df['Open'].replace(['PK','pk'],'0',inplace=True)
    df['Close'].replace(['PK','pk'],'0',inplace=True)
    df['Open_2'].replace(['PK','pk'],'0',inplace=True)
    df['Close_2'].replace(['PK','pk'],'0',inplace=True)
    df.replace('197.5u10','197.5',inplace=True) #one unique occurrence
    #map strings as floats for comparison
    df['Open'] = df['Open'].astype('float')
    df['Open_2'] = df['Open_2'].astype('float')
    df['Close'] = df['Close'].astype('float')
    df['Close_2'] = df['Close_2'].astype('float')
    #get correct over/under line at opening and closing of sportsbook
    df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
    df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
    
    #rename columns to correct home/visitor 
    df['Visitor'] = df['Team']
    df['Home'] = df['Team_2']
    
    #add season column
    df['Season'] = file[:2] + file[3:5]
    
    df['score_diff'] = df['Final'] - df['Final_2']
    df['Winner'] = df['score_diff'].apply(win_loss)
    
    #drop unneeded info
    df = df[['Date','Home','Visitor','O/U_open','O/U_close','Season','Winner']]
    return df

Use this clean_odds function to generate our dataset to combine with our scraped game data.

In [7]:
cd csv

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv


In [8]:
lines = pd.concat([clean_odds(f) for f in files])

In [9]:
lines

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner
0,20071030,SanAntonio,Portland,184.0,189.5,0708,H
1,20071030,GoldenState,Utah,214.5,212.0,0708,V
2,20071030,LALakers,Houston,191.0,199.0,0708,V
3,20071031,Toronto,Philadelphia,190.0,191.0,0708,H
4,20071031,Indiana,Washington,200.0,203.5,0708,H
...,...,...,...,...,...,...,...
1307,20190602,Toronto,GoldenState,216.0,213.5,1819,V
1308,20190605,GoldenState,Toronto,214.0,209.5,1819,V
1309,20190607,GoldenState,Toronto,216.0,215.0,1819,V
1310,20190610,Toronto,GoldenState,212.0,217.0,1819,V


In [10]:
lines.reset_index(drop=True,inplace=True)
lines

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner
0,20071030,SanAntonio,Portland,184.0,189.5,0708,H
1,20071030,GoldenState,Utah,214.5,212.0,0708,V
2,20071030,LALakers,Houston,191.0,199.0,0708,V
3,20071031,Toronto,Philadelphia,190.0,191.0,0708,H
4,20071031,Indiana,Washington,200.0,203.5,0708,H
...,...,...,...,...,...,...,...
15515,20190602,Toronto,GoldenState,216.0,213.5,1819,V
15516,20190605,GoldenState,Toronto,214.0,209.5,1819,V
15517,20190607,GoldenState,Toronto,216.0,215.0,1819,V
15518,20190610,Toronto,GoldenState,212.0,217.0,1819,V


Let's pickle this initial processing of the csv's and subsequent DataFrame. We can always comeback to this if we need it.

In [11]:
with open('lines.pickle', 'wb') as to_write:
    pickle.dump(lines, to_write)

In [12]:
pwd

'/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor/csv'

In [13]:
cd ..

/Users/jseemayer/Documents/Me/Metis/NBA-Over-Under-Predictor


In [14]:
with open('game_df.pickle','rb') as read_file:
    game_df = pickle.load(read_file)
    
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.552,.315,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.577,.301,.479,64.7,46.7,46.9,2.3,12.9,100.0,94.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.493,.404,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.523,.396,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.527,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.542,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4


In [15]:
team_abbrev = game_df['home'].unique()
team_abbrev

array(['SAS', 'GSW', 'LAL', 'IND', 'ORL', 'TOR', 'NJN', 'CLE', 'MEM',
       'NOH', 'DEN', 'MIA', 'UTA', 'SEA', 'CHA', 'ATL', 'BOS', 'MIN',
       'CHI', 'LAC', 'PHO', 'PHI', 'WAS', 'DAL', 'HOU', 'MIL', 'NYK',
       'DET', 'SAC', 'POR', 'OKC', 'BRK', 'NOP', 'CHO'], dtype=object)

In [16]:
len(team_abbrev)

34

In [17]:
lines['Home'].unique()

array(['SanAntonio', 'GoldenState', 'LALakers', 'Toronto', 'Indiana',
       'Orlando', 'NewJersey', 'Cleveland', 'Memphis', 'NewOrleans',
       'Denver', 'Miami', 'Utah', 'Seattle', 'Charlotte', 'Atlanta',
       'Boston', 'Minnesota', 'Chicago', 'Phoenix', 'LAClippers',
       'Philadelphia', 'Washington', 'Milwaukee', 'Houston', 'Dallas',
       'NewYork', 'Detroit', 'Sacramento', 'Portland', 'OklahomaCity',
       'Brooklyn'], dtype=object)

In [18]:
team_dict = {
    'SAS': 'SanAntonio',
    'GSW': 'GoldenState',
    'LAL': 'LALakers',
    'TOR': 'Toronto',
    'IND': 'Indiana',
    'ORL': 'Orlando',
    'NJN': 'NewJersey',
    'CLE': 'Cleveland',
    'MEM': 'Memphis',
    'NOH': 'NewOrleans',
    'NOP': 'NewOrleans',
    'DEN': 'Denver',
    'MIA': 'Miami',
    'UTA': 'Utah',
    'SEA': 'Seattle',
    'CHA': 'Charlotte',
    'CHO': 'Charlotte',
    'ATL': 'Atlanta',
    'BOS': 'Boston',
    'MIN': 'Minnesota',
    'CHI': 'Chicago',
    'PHO': 'Phoenix',
    'LAC': 'LAClippers',
    'PHI': 'Philadelphia',
    'WAS': 'Washington',
    'MIL': 'Milwaukee',
    'HOU': 'Houston',
    'DAL': 'Dallas',
    'NYK': 'NewYork',
    'DET': 'Detroit',
    'SAC': 'Sacramento',
    'POR': 'Portland',
    'OKC': 'OklahomaCity',
    'BRK': 'Brooklyn'
}

In [19]:
game_df['home'].map(team_dict)

0         SanAntonio
1        GoldenState
2           LALakers
3            Indiana
4            Orlando
            ...     
15516        Toronto
15517    GoldenState
15518    GoldenState
15519        Toronto
15520    GoldenState
Name: home, Length: 15521, dtype: object

In [20]:
game_df['home_team'] = game_df['home'].map(team_dict)

In [21]:
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.315,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.301,.479,64.7,46.7,46.9,2.3,12.9,100.0,94.2,Orlando
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.404,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.396,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto


In [22]:
game_df['game_date'] = game_df['game_id'].apply(lambda x: x[:8])

In [23]:
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana,20071031
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,.479,64.7,46.7,46.9,2.3,12.9,100.0,94.2,Orlando,20071031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto,20190602
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState,20190605
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610


In [24]:
game_df['id'] = game_df['game_date'] + game_df['home_team']

In [25]:
game_df

,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,...,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date,id
0,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,...,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,20071030SanAntonio
1,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,...,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,20071030GoldenState
2,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,...,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030,20071030LALakers
3,200710310IND,229,WAS,IND,98.4,98.4,.394,.484,13.2,13.6,...,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana,20071031,20071031Indiana
4,200710310ORL,185,MIL,ORL,88.1,88.1,.402,.521,12.6,10.2,...,64.7,46.7,46.9,2.3,12.9,100.0,94.2,Orlando,20071031,20071031Orlando
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15516,201906020TOR,213,GSW,TOR,100.0,100.0,.543,.431,14.0,12.5,...,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto,20190602,20190602Toronto
15517,201906050GSW,232,TOR,GSW,99.6,99.6,.628,.462,13.3,11.8,...,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState,20190605,20190605GoldenState
15518,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,...,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607,20190607GoldenState
15519,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,...,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610,20190610Toronto


In [26]:
lines['id'] = lines['Date'] + lines['Home']

In [27]:
lines

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner,id
0,20071030,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio
1,20071030,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState
2,20071030,LALakers,Houston,191.0,199.0,0708,V,20071030LALakers
3,20071031,Toronto,Philadelphia,190.0,191.0,0708,H,20071031Toronto
4,20071031,Indiana,Washington,200.0,203.5,0708,H,20071031Indiana
...,...,...,...,...,...,...,...,...
15515,20190602,Toronto,GoldenState,216.0,213.5,1819,V,20190602Toronto
15516,20190605,GoldenState,Toronto,214.0,209.5,1819,V,20190605GoldenState
15517,20190607,GoldenState,Toronto,216.0,215.0,1819,V,20190607GoldenState
15518,20190610,Toronto,GoldenState,212.0,217.0,1819,V,20190610Toronto


In [28]:
lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15520 entries, 0 to 15519
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       15520 non-null  object 
 1   Home       15520 non-null  object 
 2   Visitor    15520 non-null  object 
 3   O/U_open   15520 non-null  float64
 4   O/U_close  15520 non-null  float64
 5   Season     15520 non-null  object 
 6   Winner     15520 non-null  object 
 7   id         15520 non-null  object 
dtypes: float64(2), object(6)
memory usage: 970.1+ KB


In [29]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15521 entries, 0 to 15520
Data columns (total 71 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   game_id       15521 non-null  object
 1   total         15521 non-null  int64 
 2   away          15521 non-null  object
 3   home          15521 non-null  object
 4   pace_v        15521 non-null  object
 5   pace_h        15521 non-null  object
 6   eFg_v         15521 non-null  object
 7   eFg_h         15521 non-null  object
 8   tov_v         15521 non-null  object
 9   tov_h         15521 non-null  object
 10  orb_v         15521 non-null  object
 11  orb_h         15521 non-null  object
 12  ft_fga_v      15521 non-null  object
 13  ft_fga_h      15521 non-null  object
 14  ortg_v        15521 non-null  object
 15  ortg_h        15521 non-null  object
 16  fg_v          15521 non-null  object
 17  fga_v         15521 non-null  object
 18  fg_per_v      15521 non-null  object
 19  thre

Now both databases have a column to merge on. The 'id' column.

In [30]:
df = pd.merge(lines,game_df,on='id')

In [31]:
df

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner,id,game_id,total,...,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date
0,20071030,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio,200710300SAS,203,...,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030
1,20071030,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState,200710300GSW,213,...,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030
2,20071030,LALakers,Houston,191.0,199.0,0708,V,20071030LALakers,200710300LAL,188,...,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030
3,20071031,Toronto,Philadelphia,190.0,191.0,0708,H,20071031Toronto,200710310TOR,203,...,.200,64.3,44.6,56.1,8.8,6.2,100.0,106.8,Toronto,20071031
4,20071031,Indiana,Washington,200.0,203.5,0708,H,20071031Indiana,200710310IND,229,...,.391,64.6,50.0,52.6,11.0,7.6,100.0,101.2,Indiana,20071031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,20190602,Toronto,GoldenState,216.0,213.5,1819,V,20190602Toronto,201906020TOR,213,...,.277,85.0,53.8,48.6,8.0,4.2,100.0,109.0,Toronto,20190602
15011,20190605,GoldenState,Toronto,214.0,209.5,1819,V,20190605GoldenState,201906050GSW,232,...,.330,84.8,50.6,69.4,8.0,6.8,100.0,123.5,GoldenState,20190605
15012,20190607,GoldenState,Toronto,216.0,215.0,1819,V,20190607GoldenState,201906070GSW,197,...,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607
15013,20190610,Toronto,GoldenState,212.0,217.0,1819,V,20190610Toronto,201906100TOR,211,...,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610


In [32]:
df.shape

(15015, 78)

Drop columns that share the same information.

In [ ]:
df.drop(columns=['Home','Visitor','game_id','home_team','game_date'],inplace=True)

In [ ]:
df

Getting the difference between the line and the outcome of a particular game in order to classify our game as an "Over" or "Under".

In [33]:
df['ou1'] = df['total'] - df['O/U_open']
df['ou2'] = df['total'] - df['O/U_close']

Function to label our target based off information we have in our DataFrame, and then applying that function to our newly created columns that currently classify our games based on a positive or negative value. This simply puts a more general categorical label over that more granular scalar indicator.

In [34]:
def over_under(ou):
    if ou > 0:
        return 1
    elif ou == 0:
        return 2
    else:
        return 0

In [35]:
df['Over/Under_open'] = df['ou1'].apply(over_under)
df['Over/Under_close'] = df['ou2'].apply(over_under)

CRITICAL ASSUMPTION: Making the decision to drop "pushes". A "push" is when a betting line is hit exactly, and no money exchanges hands. In the context of this project, a total could be at 212 points, and if the game finishes with exactly 212 points, then every bettor gets their money back, on both sides (over/under 212), and the sportsbook doesn't collect anything.

In [36]:
df = df[df['Over/Under_open'] != 2]
df = df[df['Over/Under_close'] != 2]

About 400 instances of a "push", or about 2.7% of our original dataset. This number can be remembered for sampling or simulation purposes later.

Convert date to pd.datetime object. Makes plotting the time series compatible with matplotlib, and building season dictionary later.

In [37]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [38]:
df['ou2'].describe()

count    14622.000000
mean         0.302011
std         17.763216
min        -67.500000
25%        -11.500000
50%         -0.500000
75%         11.500000
max         99.500000
Name: ou2, dtype: float64

In [ ]:
build kde plots of ou movement throughout time

### Stat building (feature engineering)

We want to ultimately end up with a model that takes both teams' recent performances and can make a prediction on the total, so we can make a new column to hold home/visitor every other row, and then rename the _h and _v columns for self, opponent.

In [39]:
d = df.reindex(df.index.repeat(2)).reset_index(drop=True)

Now that we have every game repeated, let's create a column to indicate which team these self/opponent stats represent. We can do this by creating an empty column and selectively copy from the 'home' and 'away' columns that repeat across the 2 rows. Then we'll do that same for the opponent.

In [40]:
d['team'] = np.nan
#home team will be all even indexes of this dataset
d['team'][::2] = d['home'][::2]
#away team will be all odd indexes of this dataset
d['team'][1::2] = d['away'][1::2]

<ipython-input-40-d36efb69a2a0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['team'][::2] = d['home'][::2]
<ipython-input-40-d36efb69a2a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['team'][1::2] = d['away'][1::2]


In [41]:
d['opp'] = np.nan
d['opp'][::2] = d['away'][::2]
d['opp'][1::2] = d['home'][1::2]

<ipython-input-41-84f043bae77b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['opp'][::2] = d['away'][::2]
<ipython-input-41-84f043bae77b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['opp'][1::2] = d['home'][1::2]


Let's keep track of home/away, can't hurt. We'll use the same process as we just used. Note: this can also be used as a sanity check to make sure that our splitting worked correctly. If it did, we should see 'H', 'V' every other row.

In [42]:
d['home/away'] = np.nan
#home team will be all even indexes of this dataset
d['home/away'][::2] = 'H'
#away team will be all odd indexes of this dataset
d['home/away'][1::2] = 'V'

<ipython-input-42-915be6e0f5cd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['home/away'][::2] = 'H'
<ipython-input-42-915be6e0f5cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['home/away'][1::2] = 'V'


Now we have to map the _v and _h stats appropriate to new columns for self or opponent stats.

In [ ]:
stats = ['pace','eFg','tov','orb','ft_fga','ortg','fg','fga','fg_per','threes','threes_att','threes_per',
         'ft','fta','ft_per','drb','trb','ast','stl','blk','to','fouls','ts_per','threes_ar','ft_ar',
         'drb_per','trb_per','ast_per','stl_per','blk_per','user_per','drtg']
for stat in stats:
    d['{}'.format(stat)] = ""
    d['{}'.format(stat)][::2] = d['{}_h'.format(stat)][::2]
    d['{}'.format(stat)][1::2] = d['{}_v'.format(stat)][1::2]
    d['{}_opp'.format(stat)] = ""
    d['{}_opp'.format(stat)][::2] = d['{}_v'.format(stat)][::2]
    d['{}_opp'.format(stat)][1::2] = d['{}_h'.format(stat)][1::2]

Then we can drop all of our columns with '_v' & '_h' since they contain extra, and now, redundant information.

In [ ]:
mylist = []
for stat in stats:
    mylist.append('{}_v'.format(stat))
    mylist.append('{}_h'.format(stat))
#print(mylist)
d = d.drop(columns=mylist)

Now our DataFrame (d) has stats for each team and their opponent for every game. Now let's get some rolling averages. First we'll create an empty column that we'll populate with a rolling count for every time that team has appeared. This will get us the number of games each team has played up INCLUDING that game for each season. We can use that info to incorporate some rolling averages.

In [43]:
d['team_season'] = d['team'] + d['Season']

In [44]:
d['game_num'] = d.groupby('team_season').cumcount()+1

Now we are ready to compute some rolling average's in a certain season. Remember, we want to get a snapshot of how both team's have perfromed recently (last 5 games), and get a classification from this info. So let's go back to our method of creating empty columns we can assign data to.

In [ ]:
for stat in d.iloc[:,15:79]:
    d['{}_rolling'.format(stat)] = np.nan

Cast our stat columns as floats to perform some operations on them.

In [ ]:
for col in d.iloc[:,15:79].columns:
    d[col] = d[col].astype('float')

This may not be the most pythonic way to populate these new columns, but it works, and doesn't take long. At a high level, we are going team by team, then season by season, for that team, and applying 5 game rolling averages to their stats and opponent stats.

In [ ]:
for team in d['team'].unique():
    mask = d['team'] == team
    d4 = d[mask]
    for season in d4['Season'].unique():
        mask = d4['Season'] == '{}'.format(season)
        d5 = d4[mask]
        for stat in list(d5.iloc[:,15:79].columns):
            d5['{}_rolling'.format(stat)] = d5.rolling(window=5)['{}'.format(stat)].mean().shift(1)
        d.update(d5)

IMPORTANT NOTE: this DataFrame contains rows with NaN's. Every game that is in the first 5 of the season has no rolling averages, this can be problematic when we take this DataFrame and try to feed it into a sk-learn ML model. We can address this now, by simply dropping these games(~5% of our dataset), or apply the previous season's median rolling average. This is where some decisions need to be made, and I would advise just dropping the games for 2 reasons. One, taking out 5 games won't change the application our model will have as there are 67-75 games left in the year we can apply this model to once in production. Two, a median of the rolling averages, or any other kind of congregation statistic being applied over different seasons is mostly likely not a sound choice in light of team personnel turnover, from retirings, trades, the draft, and free agent moves; not to mention new coaching staffs. This could be applicable to a few teams (ones that experience little of this roster turnover), but not enough to apply that thinking across the entire dataset.

In [46]:
pd.set_option("display.max_columns", 300)

In [47]:
d

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner,id,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,orb_v,orb_h,ft_fga_v,ft_fga_h,ortg_v,ortg_h,fg_v,fga_v,fg_per_v,threes_v,threes_att_v,threes_per_v,ft_v,fta_v,ft_per_v,drb_v,trb_v,ast_v,stl_v,blk_v,to_v,fouls_v,ts_per_v,threes_ar_v,ft_ar_v,drb_per_v,trb_per_v,ast_per_v,stl_per_v,blk_per_v,user_per_v,drtg_v,fg_h,fga_h,fg_per_h,threes_h,threes_att_h,threes_per_h,ft_h,fta_h,ft_per_h,drb_h,trb_h,ast_h,stl_h,blk_h,to_h,fouls_h,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date,ou1,ou2,Over/Under_open,Over/Under_close,team,opp,home/away,team_season,game_num
0,2007-10-30,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,22.2,27.3,.167,.207,105.7,115.5,240,39,78,.500,6,13,.462,13,17,8,32,40,15,1,4,16,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5,240,41,87,.471,6,24,.250,18,26,12,28,40,21,8,4,8,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,19.0,13.5,1,1,SAS,POR,H,SAS0708,1
1,2007-10-30,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,22.2,27.3,.167,.207,105.7,115.5,240,39,78,.500,6,13,.462,13,17,8,32,40,15,1,4,16,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5,240,41,87,.471,6,24,.250,18,26,12,28,40,21,8,4,8,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,19.0,13.5,1,1,POR,SAS,V,POR0708,1
2,2007-10-30,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,34.8,14.9,.333,.338,111.3,91.3,240,41,90,.456,5,11,.455,30,36,16,40,56,24,9,7,19,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3,240,32,77,.416,6,23,.261,26,38,7,30,37,19,8,9,20,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,-1.5,1.0,0,1,GSW,UTA,H,GSW0708,1
3,2007-10-30,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,34.8,14.9,.333,.338,111.3,91.3,240,41,90,.456,5,11,.455,30,36,16,40,56,24,9,7,19,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3,240,32,77,.416,6,23,.261,26,38,7,30,37,19,8,9,20,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,-1.5,1.0,0,1,UTA,GSW,V,UTA0708,1
4,2007-10-30,LALakers,Houston,191.0,199.0,0708,V,20071030LALakers,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,31.6,22.9,.284,.355,102.1,99.9,240,34,74,.459,6,22,.273,21,31,12,37,49,23,10,5,18,.542,.297,.419,77.1,57.0,67.6,10.7,7.4,100.0,99.9,240,32,76,.421,2,8,.250,27,45,11,26,37,18,16,3,12,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030,-3.0,-11.0,0,0,LAL,HOU,H,LAL0708,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29239,2019-06-07,GoldenState,Toronto,216.0,215.0,1819,V,20190607GoldenState,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,17.1,20.0,.267,.179,110.7,97.0,240,36,86,.419,10,32,.313,23,24,7,32,39,22,12,4,9,.544,.372,.279,80.0,48.1,61.1,12.7,7.8,100.0,97.0,240,35,78,.449,8,27,.296,14,21,8,34,42,26,6,6,17,.527,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607,-19.0,-18.0,0,0,TOR,GSW,V,TOR1819,99
29240,2019-06-10,Toronto,GoldenState,212.0,217.0,1819,V,20190610Toronto,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,16.7,29.5,.122,.247,112.4,111.3,240,38,82,.463,20,42,.476,10,14,6,31,37,27,5,7,15,.601,.512,.171,70.5,46.3,71.1,5.3,13.2,100.0,111.3,240,38,85,.447,8,32,.250,21,27,13,30,43,19,6,5,13,.542,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610,-1.0,-6.0,0,0,TOR,GSW,H,TOR1819,100
29241,2019-06-10,Toronto,GoldenState,212.0,217.0,1819,V,20190610Toronto,2019

In [76]:
d.drop(columns='W',inplace=True)

In [78]:
result = d['Winner']
mylist = []
for i in range(len(result)):
    if result[i] == d['home/away'][i]:
        mylist.append(1)
    else:
        mylist.append(0)
d['W'] = pd.Series(mylist)
d['W']

0        1
1        0
2        0
3        1
4        0
        ..
29239    1
29240    0
29241    1
29242    0
29243    1
Name: W, Length: 29244, dtype: int64

In [80]:
d

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner,id,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,orb_v,orb_h,ft_fga_v,ft_fga_h,ortg_v,ortg_h,fg_v,fga_v,fg_per_v,threes_v,threes_att_v,threes_per_v,ft_v,fta_v,ft_per_v,drb_v,trb_v,ast_v,stl_v,blk_v,to_v,fouls_v,ts_per_v,threes_ar_v,ft_ar_v,drb_per_v,trb_per_v,ast_per_v,stl_per_v,blk_per_v,user_per_v,drtg_v,fg_h,fga_h,fg_per_h,threes_h,threes_att_h,threes_per_h,ft_h,fta_h,ft_per_h,drb_h,trb_h,ast_h,stl_h,blk_h,to_h,fouls_h,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date,ou1,ou2,Over/Under_open,Over/Under_close,team,opp,home/away,team_season,game_num,W
0,2007-10-30,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,22.2,27.3,.167,.207,105.7,115.5,240,39,78,.500,6,13,.462,13,17,8,32,40,15,1,4,16,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5,240,41,87,.471,6,24,.250,18,26,12,28,40,21,8,4,8,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,19.0,13.5,1,1,SAS,POR,H,SAS0708,1,1
1,2007-10-30,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,22.2,27.3,.167,.207,105.7,115.5,240,39,78,.500,6,13,.462,13,17,8,32,40,15,1,4,16,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5,240,41,87,.471,6,24,.250,18,26,12,28,40,21,8,4,8,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,19.0,13.5,1,1,POR,SAS,V,POR0708,1,0
2,2007-10-30,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,34.8,14.9,.333,.338,111.3,91.3,240,41,90,.456,5,11,.455,30,36,16,40,56,24,9,7,19,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3,240,32,77,.416,6,23,.261,26,38,7,30,37,19,8,9,20,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,-1.5,1.0,0,1,GSW,UTA,H,GSW0708,1,0
3,2007-10-30,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,34.8,14.9,.333,.338,111.3,91.3,240,41,90,.456,5,11,.455,30,36,16,40,56,24,9,7,19,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3,240,32,77,.416,6,23,.261,26,38,7,30,37,19,8,9,20,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,-1.5,1.0,0,1,UTA,GSW,V,UTA0708,1,1
4,2007-10-30,LALakers,Houston,191.0,199.0,0708,V,20071030LALakers,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,31.6,22.9,.284,.355,102.1,99.9,240,34,74,.459,6,22,.273,21,31,12,37,49,23,10,5,18,.542,.297,.419,77.1,57.0,67.6,10.7,7.4,100.0,99.9,240,32,76,.421,2,8,.250,27,45,11,26,37,18,16,3,12,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030,-3.0,-11.0,0,0,LAL,HOU,H,LAL0708,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29239,2019-06-07,GoldenState,Toronto,216.0,215.0,1819,V,20190607GoldenState,201906070GSW,197,TOR,GSW,94.8,94.8,.477,.500,8.5,16.3,17.1,20.0,.267,.179,110.7,97.0,240,36,86,.419,10,32,.313,23,24,7,32,39,22,12,4,9,.544,.372,.279,80.0,48.1,61.1,12.7,7.8,100.0,97.0,240,35,78,.449,8,27,.296,14,21,8,34,42,26,6,6,17,.527,.346,.269,82.9,51.9,74.3,6.3,11.1,100.0,110.7,GoldenState,20190607,-19.0,-18.0,0,0,TOR,GSW,V,TOR1819,99,1
29240,2019-06-10,Toronto,GoldenState,212.0,217.0,1819,V,20190610Toronto,201906100TOR,211,GSW,TOR,94.3,94.3,.585,.494,14.5,11.8,16.7,29.5,.122,.247,112.4,111.3,240,38,82,.463,20,42,.476,10,14,6,31,37,27,5,7,15,.601,.512,.171,70.5,46.3,71.1,5.3,13.2,100.0,111.3,240,38,85,.447,8,32,.250,21,27,13,30,43,19,6,5,13,.542,.376,.318,83.3,53.8,50.0,6.4,12.5,100.0,112.4,Toronto,20190610,-1.0,-6.0,0,0,TOR,GSW,H,TOR1819,100,0
29241,2019-06-10,Toronto,GoldenState,212.0,217.0,1819,V,

We won't cut off these rows in this notebook, but can easily do so in our modeling notebook by using the following mask and declaring this filtered d as our new d.

In [ ]:
d[d['game_num']>5]

The final thing we have to do before we can begin classifying our games is make each row one game. Our features will include rolling averages for each of our team's and opponent's stats. From here we can get a baseline sense if our logic, of recent past performances, is good basis for classification.

In [ ]:
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 101)

In [79]:
df = d.merge(d, on='id', suffixes=(None,'_v')) #gets games to be in one row (creates 4/game)
#df.tail(8)
df = df[1::4] #gets correct mapping
df.reset_index(drop=True, inplace=True)

In [81]:
df

,Date,Home,Visitor,O/U_open,O/U_close,Season,Winner,id,game_id,total,away,home,pace_v,pace_h,eFg_v,eFg_h,tov_v,tov_h,orb_v,orb_h,ft_fga_v,ft_fga_h,ortg_v,ortg_h,fg_v,fga_v,fg_per_v,threes_v,threes_att_v,threes_per_v,ft_v,fta_v,ft_per_v,drb_v,trb_v,ast_v,stl_v,blk_v,to_v,fouls_v,ts_per_v,threes_ar_v,ft_ar_v,drb_per_v,trb_per_v,ast_per_v,stl_per_v,blk_per_v,user_per_v,drtg_v,fg_h,fga_h,fg_per_h,threes_h,threes_att_h,threes_per_h,ft_h,fta_h,ft_per_h,drb_h,trb_h,ast_h,stl_h,blk_h,to_h,fouls_h,ts_per_h,threes_ar_h,ft_ar_h,drb_per_h,trb_per_h,ast_per_h,stl_per_h,blk_per_h,user_per_h,drtg_h,home_team,game_date,ou1,ou2,Over/Under_open,Over/Under_close,team,opp,home/away,team_season,game_num,W,Date_v,Home_v,Visitor_v,O/U_open_v,O/U_close_v,Season_v,Winner_v,game_id_v,total_v,away_v,home_v,pace_v_v,pace_h_v,eFg_v_v,eFg_h_v,tov_v_v,tov_h_v,orb_v_v,orb_h_v,ft_fga_v_v,ft_fga_h_v,ortg_v_v,ortg_h_v,fg_v_v,fga_v_v,fg_per_v_v,threes_v_v,threes_att_v_v,threes_per_v_v,ft_v_v,fta_v_v,ft_per_v_v,drb_v_v,trb_v_v,ast_v_v,stl_v_v,blk_v_v,to_v_v,fouls_v_v,ts_per_v_v,threes_ar_v_v,ft_ar_v_v,drb_per_v_v,trb_per_v_v,ast_per_v_v,stl_per_v_v,blk_per_v_v,user_per_v_v,drtg_v_v,fg_h_v,fga_h_v,fg_per_h_v,threes_h_v,threes_att_h_v,threes_per_h_v,ft_h_v,fta_h_v,ft_per_h_v,drb_h_v,trb_h_v,ast_h_v,stl_h_v,blk_h_v,to_h_v,fouls_h_v,ts_per_h_v,threes_ar_h_v,ft_ar_h_v,drb_per_h_v,trb_per_h_v,ast_per_h_v,stl_per_h_v,blk_per_h_v,user_per_h_v,drtg_h_v,home_team_v,game_date_v,ou1_v,ou2_v,Over/Under_open_v,Over/Under_close_v,team_v,opp_v,home/away_v,team_season_v,game_num_v,W_v
0,2007-10-30,SanAntonio,Portland,184.0,189.5,0708,H,20071030SanAntonio,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,22.2,27.3,.167,.207,105.7,115.5,240,39,78,.500,6,13,.462,13,17,8,32,40,15,1,4,16,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5,240,41,87,.471,6,24,.250,18,26,12,28,40,21,8,4,8,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,19.0,13.5,1,1,SAS,POR,H,SAS0708,1,1,2007-10-30,SanAntonio,Portland,184.0,189.5,0708,H,200710300SAS,203,POR,SAS,91.8,91.8,.538,.506,15.8,7.5,22.2,27.3,.167,.207,105.7,115.5,240,39,78,.500,6,13,.462,13,17,8,32,40,15,1,4,16,.567,.167,.218,72.7,50.0,38.5,1.1,6.3,100.0,115.5,240,41,87,.471,6,24,.250,18,26,12,28,40,21,8,4,8,.538,.276,.299,77.8,50.0,51.2,8.7,6.2,100.0,105.7,SanAntonio,20071030,19.0,13.5,1,1,POR,SAS,V,POR0708,1,0
1,2007-10-30,GoldenState,Utah,214.5,212.0,0708,V,20071030GoldenState,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,34.8,14.9,.333,.338,111.3,91.3,240,41,90,.456,5,11,.455,30,36,16,40,56,24,9,7,19,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3,240,32,77,.416,6,23,.261,26,38,7,30,37,19,8,9,20,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,-1.5,1.0,0,1,GSW,UTA,H,GSW0708,1,0,2007-10-30,GoldenState,Utah,214.5,212.0,0708,V,200710300GSW,213,UTA,GSW,105.1,105.1,.483,.455,15.2,17.6,34.8,14.9,.333,.338,111.3,91.3,240,41,90,.456,5,11,.455,30,36,16,40,56,24,9,7,19,.553,.122,.400,85.1,60.2,58.5,8.6,13.0,100.0,91.3,240,32,77,.416,6,23,.261,26,38,7,30,37,19,8,9,20,.512,.299,.494,65.2,39.8,59.4,7.6,11.4,100.0,111.3,GoldenState,20071030,-1.5,1.0,0,1,UTA,GSW,V,UTA0708,1,1
2,2007-10-30,LALakers,Houston,191.0,199.0,0708,V,20071030LALakers,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,31.6,22.9,.284,.355,102.1,99.9,240,34,74,.459,6,22,.273,21,31,12,37,49,23,10,5,18,.542,.297,.419,77.1,57.0,67.6,10.7,7.4,100.0,99.9,240,32,76,.421,2,8,.250,27,45,11,26,37,18,16,3,12,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030,-3.0,-11.0,0,0,LAL,HOU,H,LAL0708,1,0,2007-10-30,LALakers,Houston,191.0,199.0,0708,V,200710300LAL,188,HOU,LAL,93.0,93.0,.500,.434,17.0,11.1,31.6,22.9,.284,.355,102.1,99.9,240,34,74,.459,6,22,.273,21,31,12,37,49,23,10,5,18,.542,.297,.419,77.1,57.0,67.6,10.7,7.4,100.0,99.9,240,32,76,.421,2,8,.250,27,45,11,26,37,18,16,3,12,.485,.105,.592,68.4,43.0,56.3,17.2,5.8,100.0,102.1,LALakers,20071030,-3.0,-11.0,0,0,HOU,LAL,V,HOU0708,1,1
3,2007-10-31,Toronto,Philadelphia,190.

Now that our data is fully processed, we are ready to process this in a classification algorithm

In [82]:
with open('d_rolling.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

## Scratch Work

In [ ]:
cd csv

In [ ]:
df = pd.read_csv(files[-1])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.iloc[:,:13]

In [ ]:
df.info()

In [ ]:
df.shape[0]

In [ ]:
mylist = []
for i in range(1, int(df.shape[0]/2 + 1)):
    mylist.append(i)
    mylist.append(i)
print(mylist[0],mylist[-1])

In [ ]:
len(range(1,int(df.shape[0]/2 + 1)))

In [ ]:
df['game_id'] = mylist

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.info()

In [ ]:
df = df.merge(df, on='game_id', suffixes=(None,'_2'))

In [ ]:
df

In [ ]:
df = df[1::4]
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
#sanity check that every game only appears once
df['game_id']

In [ ]:
df['Date'] = df['Date'].astype('int')
df['Date']

In [ ]:
df['Date'] = df['Date'].astype('str')
df['Date']

In [ ]:
df['Date'] = df['Date'].str.strip('.0')

In [ ]:
df['Date'].head

In [ ]:
dates = []
for date in df['Date']:
    if int(date) > 1000:
        date = '20'+files[2][:2]+date
        dates.append(date)
    else:
        date = '20'+files[2][3:5]+'0'+date
        dates.append(date)
df['Date'] = dates

In [ ]:
df['Date']

In [ ]:
df

In [ ]:
df['Visitor'] = df['Team']
df['Home'] = df['Team_2']
df

In [ ]:
df['Open'].replace('pk','0',inplace=True)
df['Close'].replace('pk','0',inplace=True)
df['Open_2'].replace('pk','0',inplace=True)
df['Close_2'].replace('pk','0',inplace=True)
df.replace('197.5u10','197.5',inplace=True)

In [ ]:
mask = df['Open'].values == 'pk'
df[mask]

In [ ]:
df['Open'] = df['Open'].astype('float')
df['Open_2'] = df['Open_2'].astype('float')
df['Close'] = df['Close'].astype('float')
df['Close_2'] = df['Close_2'].astype('float')

In [ ]:
df['O/U_open'] = np.where(df['Open'] > df['Open_2'],df['Open'],df['Open_2'])
df

In [ ]:
df['O/U_close'] = np.where(df['Close'] > df['Close_2'],df['Close'],df['Close_2'])
df

Sanity check that opening and closing lines operations worked successfully.
So long as numbers are around 200, we know we are ok.

In [ ]:
df['O/U_open'].min()

In [ ]:
df['O/U_close'].min()

Determine Winner/Loser each game

In [ ]:
df['score_diff'] = df['Final'] - df['Final_2']
df['score_diff']

In [ ]:
def win_loss(score):
    if score > 0:
        return 'V'
    else:
        return 'H'

In [ ]:
df['Winner'] = df['score_diff'].apply(win_loss)

In [ ]:
df['Winner']

In [ ]:
df = df[['Date','Home','Visitor','O/U_open','O/U_close']]
df

### Processing our lines df

First, and most importantly, we must create our target labels. This will be done by comparing the total the the opening and closing lines, and mapping that result to one of three categories: Over, Under, or Push. This will represent the winning result of that game.

In [ ]:
df

In [ ]:
df['ou1'] = df['total'] - df['O/U_open']
df['ou2'] = df['total'] - df['O/U_close']

In [ ]:
df

In [ ]:
def over_under(ou):
    if ou > 0:
        return 1
    elif ou == 0:
        return 2
    else:
        return 0

In [ ]:
df['Over/Under_open'] = df['ou1'].apply(over_under)
df['Over/Under_close'] = df['ou2'].apply(over_under)

In [ ]:
df

Making the decision to drop pushes as no money exchanges hands in this scenerio.

In [ ]:
df = df[df['Over/Under_open'] != 2]
df = df[df['Over/Under_close'] != 2]

In [ ]:
df

About 400 instances of a push, or about 2.7% of our original dataset. This number can be remembered for sampling or simulation purposes later.

### Stat Building

We have the same set of stats for both teams in any one game, so we can build offense/defense for both teams.

In [ ]:
'''map stats accordingly:
1. Get 2 "sets" of stats per game:

visitor: offense - _v stat avgs heading into the game; defense - _h stat avgs heading into the game
home: offese - _h stat avgs heading into the game; defense - _v stat avgs heading into the game

2. Map visitor/home stats to respective teams

3. Build dictionary of team's seasons to be further processed.

{1415: {GSW: {..game_35:{offense/defense stats},game_36:{..}

4. Process dictionary to have more stats/potential model features:

{GSW: {..game_35:{offense/defense stats averaged through 34 games},game_36:{..}

5. ?

'''

In [ ]:
df.info()

Convert date to pd.datetime object. May help with building season dictionary later.

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

In [ ]:
df['Date']

In [ ]:
d = df.reindex(df.index.repeat(2)).reset_index(drop=True)

Now that we have every game twice, we can make a new column to hold home/visitor every other row, and then rename the _h and _v columns for self, opponent.

In [ ]:
d

Let's create the column we want to place our values, then selectively copy.

In [ ]:
d['team'] = np.nan

In [ ]:
#home team will be all even indexes of this dataset
d['team'][::2] = d['home'][::2]

In [ ]:
#away team will be all odd indexes of this dataset
d['team'][1::2] = d['away'][1::2]

In [ ]:
d['opp'] = np.nan

In [ ]:
d['opp'][::2] = d['away'][::2]
d['opp'][1::2] = d['home'][1::2]

In [ ]:
d[['team','opp']]

Let's keep track of home/away, can't hurt. We'll use the same process as we just used.

In [ ]:
d['home/away'] = np.nan
#home team will be all even indexes of this dataset
d['home/away'][::2] = 'H'
#away team will be all odd indexes of this dataset
d['home/away'][1::2] = 'V'

In [ ]:
d.info()

Now we have to map the _v and _h stats appropriate to new columns for self or opponent stats.

In [ ]:
stats = ['pace','eFg','tov','orb','ft_fga','ortg','fg','fga','fg_per','threes','threes_att','threes_per',
         'ft','fta','ft_per','drb','trb','ast','stl','blk','to','fouls','ts_per','threes_ar','ft_ar',
         'drb_per','trb_per','ast_per','stl_per','blk_per','user_per','drtg']
for stat in stats:
    d['{}'.format(stat)] = ""
    d['{}'.format(stat)][::2] = d['{}_h'.format(stat)][::2]
    d['{}'.format(stat)][1::2] = d['{}_v'.format(stat)][1::2]
    d['{}_opp'.format(stat)] = ""
    d['{}_opp'.format(stat)][::2] = d['{}_v'.format(stat)][::2]
    d['{}_opp'.format(stat)][1::2] = d['{}_h'.format(stat)][1::2]

In [ ]:
mylist = []
for stat in stats:
    mylist.append('{}_v'.format(stat))
    mylist.append('{}_h'.format(stat))
#print(mylist)
d = d.drop(columns=mylist)

In [ ]:
d.info()

Now our DataFrame (d) has stats for each team and their opponent for every game. Now let's get some rolling averages.

First we'll create an empty column that we'll populate with a rolling count for every time that team has appeared. This will get us the number of games each team has played up INCLUDING that game for each season. We can use that info to incorporate some rolling averages.

In [ ]:
d['team_season'] = d['team'] + d['Season']

In [ ]:
d

In [ ]:
d['game_num'] = d.groupby('team_season').cumcount()+1
d

In [ ]:
d.info()

Cast our stat columns as floats to perform some operations on them.

In [ ]:
for col in d.iloc[:,15:79].columns:
    d[col] = d[col].astype('float')

In [ ]:
d.info()

In [ ]:
d['team_season_game_num'] = d['team_season'] + d['game_num'].astype('str')

In [ ]:
d['team_season_game_num']

In [ ]:
for stat in d.iloc[:,15:79]:
    d['{}_rolling'.format(stat)] = np.nan

In [ ]:
d.info()

In [ ]:
for team in d['team'].unique():
    mask = d['team'] == team
    d4 = d[mask]
    for season in d4['Season'].unique():
        mask = d4['Season'] == '{}'.format(season)
        d5 = d4[mask]
        for stat in list(d5.iloc[:,15:79].columns):
            d5['{}_rolling'.format(stat)] = d5.rolling(window=5)['{}'.format(stat)].mean().shift(1)
        d.update(d5)

In [ ]:
d[d['game_num']>5]

For each team, get a single season, calculate rolling averages, then insert into our DataFrame (d). Necessary since every season has potentially different number of games

In [ ]:
for season in d4['Season'].unique():
    mask = d4['Season'] == '{}'.format(season)
    d5 = d4[mask]
    d5
    for stat in list(d5.iloc[:,15:79].columns):
        d5['{}_rolling'.format(stat)] = d5.rolling(window=5)['{}'.format(stat)].mean().shift(1)
    #print(len(d4[d4['Season'] == '{}'.format(season)]))

In [ ]:
d5.columns[82:146]

In [ ]:
d5.index.values

In [ ]:
d5

In [ ]:
d.update(d5)

In [ ]:
d.iloc[28947:28970,:]

In [ ]:
stat_list = list(d3.iloc[:,5:].columns)

In [ ]:
for stat in stat_list:
    d3['{}_rolling'.format(stat)] = d3.rolling(window=5)['{}'.format(stat)].mean().shift(1)

In [ ]:
d3

## Scratch Work Graveyard

Processing to merge datasets graveyard

In [ ]:
#tried self-merging on game_id, but this is only applicable if in different columns
#df1 = df1.merge(
#            right=df1[opp_pull_cols],
#            left_on=["game_id", "team"],
#            right_on=["game_id", "opp"],
#            suffixes=[None, "_opp"],
#        )

In [ ]:
#tried to use drop_duplicates method but can only keep first or last
#df.drop_duplicates(subset=['game_id'], keep='second')

Rolling average graveyard

In [ ]:
#found online, talks about rolling average based on multiple columns, never tried
#df.loc[:, 'value_sma_10'] = df.groupby(by='object')[['object', 'period']].rolling(window=10, min_periods=1, on='period').mean().reset_index(level='object')['value']

In [ ]:
#found online, got it to work, but not quite applicable to this situation
#span = 5
#sma = d2.rolling(window=span, min_periods=span).mean()[:span]
#rest = d2[span:]
#pd.concat([sma, rest]).ewm(span=span, adjust=False).mean()

In [ ]:
#didn't work - ValueError: cannot reindex from a duplicate axis
#d2['eFg_rolling'] = d2.groupby(['team_season','game_num'])['eFg'].rolling(10).mean().droplevel(level=[0])

In [ ]:
#df1 = d.copy()
#df1

In [ ]:
#df1 = d.groupby(['team_season','game_num']).rolling(5)['eFg'].mean().reset_index(drop=True)

In [ ]:
#df1['pace_rolling'] = d.groupby(['team_season','game_num'])[5:,'pace'].transform(lambda x: x.rolling(10, 10).mean())